# Setup S3 Buckets

In [4]:
# List the S3 public bucket content
!aws s3 ls s3://ads508-group7/

                           PRE Unsaved/
2025-03-15 21:02:43     546709 2023_Registered_Foreclosure_Properties.csv
2025-03-15 21:02:44     552165 2024_Registered_Foreclosure_Properties.csv
2025-03-15 21:40:42   92038326 Arrest_Data_from_2020_to_Present.csv
2025-03-15 21:40:42  255509498 Crime_Data_from_2020_to_Present.csv


In [5]:
import boto3
import sagemaker
import pandas as pd

# create sagemaker session
sess =  sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [6]:
# Set S3 Source Location (Public S3 Bucket)
s3_public_path = "s3://ads508-group7"

In [7]:
%store s3_public_path

Stored 's3_public_path' (str)


In [8]:
# Set S3 Destination Location (Private S3 Bucket)
s3_private_path = "s3://{}/ads508-group7".format(bucket)
print(s3_private_path)

s3://sagemaker-us-east-1-643862913351/ads508-group7


In [9]:
%store s3_private_path

Stored 's3_private_path' (str)


In [10]:
#Copy Data From the Public S3 Bucket to our Private S3 Bucket in this Account

In [11]:
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "2023_Registered_Foreclosure_Properties.csv"
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "2024_Registered_Foreclosure_Properties.csv"
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "Arrest_Data_from_2020_to_Present.csv"
!aws s3 cp --recursive $s3_public_path/ $s3_private_path/ --exclude "*" --include "Crime_Data_from_2020_to_Present.csv"

copy: s3://ads508-group7/2023_Registered_Foreclosure_Properties.csv to s3://sagemaker-us-east-1-643862913351/ads508-group7/2023_Registered_Foreclosure_Properties.csv
copy: s3://ads508-group7/2024_Registered_Foreclosure_Properties.csv to s3://sagemaker-us-east-1-643862913351/ads508-group7/2024_Registered_Foreclosure_Properties.csv
copy: s3://ads508-group7/Arrest_Data_from_2020_to_Present.csv to s3://sagemaker-us-east-1-643862913351/ads508-group7/Arrest_Data_from_2020_to_Present.csv
copy: s3://ads508-group7/Crime_Data_from_2020_to_Present.csv to s3://sagemaker-us-east-1-643862913351/ads508-group7/Crime_Data_from_2020_to_Present.csv


In [12]:
# List Files in our Private S3 Bucket in this Account
print(s3_private_path)

s3://sagemaker-us-east-1-643862913351/ads508-group7


In [13]:
!aws s3 ls $s3_private_path/

                           PRE arrests/
2025-03-21 01:01:57     546709 2023_Registered_Foreclosure_Properties.csv
2025-03-21 01:01:59     552165 2024_Registered_Foreclosure_Properties.csv
2025-03-21 01:02:01   92038326 Arrest_Data_from_2020_to_Present.csv
2025-03-21 01:02:04  255509498 Crime_Data_from_2020_to_Present.csv


In [14]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-{}-{}/ads508-group7/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

/tmp/ipykernel_151/288684959.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [15]:
# store variables for the next notebook
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path                            -> 's3://sagemaker-us-east-1-643862913351/ads508-grou
s3_public_path                             -> 's3://ads508-group7'


# Setup Athena Database

## Create Database

In [25]:
#setup pyathena
!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

In [26]:
database_name = "crime_foreclosure_db"
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS crime_foreclosure_db


In [27]:
pd.read_sql(statement, conn)

/tmp/ipykernel_151/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


## Validate Database Set Up Correctly

In [28]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_151/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,crime_foreclosure_db
1,default


In [29]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [30]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [31]:
# Store Variables for the Next Notebooks
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path                            -> 's3://sagemaker-us-east-1-643862913351/ads508-grou
s3_public_path                             -> 's3://ads508-group7'


## Create tables in Athena

### Arrest Data

In [61]:
#used ChatGPT for SERDEPROPERTIES refinement. Recieved errors for nulls, ChatGPT used to troubleshoot.
from pyathena import connect

# connection
database_name = "crime_foreclosure_db"
table_name = "arrests"
s3_data_location = "s3://sagemaker-us-east-1-643862913351/ads508-group7/arrests/"
s3_staging_dir = "s3://sagemaker-us-east-1-643862913351/ads508-group7/query-results/"  # Athena’s output
conn = connect(
    s3_staging_dir=s3_staging_dir,
    region_name="us-east-1"
)
cursor = conn.cursor()

#SQL Create table, created all as strings to ensure everything loaded. Received errors with int. 
statement = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
    report_id string,
    report_type string,
    arrest_date string,
    time string,
    area_id string,
    area_name string,
    reporting_district string,
    age string,
    sex_code string,
    descent_code string,
    charge_group_code string,
    charge_group_description string,
    arrest_type_code string,
    charge string,
    charge_description string,
    disposition_description string,
    address string,
    cross_street string,
    lat string,
    lon string,
    location string,
    booking_date string,
    booking_time string,
    booking_location string,
    booking_location_code string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  'separatorChar' = ',',
  'quoteChar'     = '\"', 
  'serialization.null.format' = ''

)
LOCATION '{s3_data_location}'
TBLPROPERTIES ('skip.header.line.count'='1')
"""

# execute sql
cursor.execute(statement)

In [59]:
#validate table created
statement = "SHOW TABLES IN crime_foreclosure_db"
df_show = pd.read_sql(statement, conn)
df_show

/tmp/ipykernel_151/2343679411.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,arrests


In [60]:
#validate that data loaded
query = "SELECT * FROM crime_foreclosure_db.arrests LIMIT 5"
arrest_sample = pd.read_sql(query, conn)
arrest_sample

/tmp/ipykernel_151/3842108231.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  arrest_sample = pd.read_sql(query, conn)


,report_id,report_type,arrest_date,time,area_id,area_name,reporting_district,age,sex_code,descent_code,...,disposition_description,address,cross_street,lat,lon,location,booking_date,booking_time,booking_location,booking_location_code
0,6185369,BOOKING,05/24/2021 12:00:00 AM,1400,05,Harbor,0585,29,M,H,...,OTHER (REQUIRES ADDITION ACTION),500 W 21ST ST,,33.7252,-118.2885,POINT (-118.2885 33.7252),05/24/2021 12:00:00 AM,1613,77TH ST,4212
1,221900608,RFC,04/17/2022 12:00:00 AM,0900,19,Mission,1998,37,M,O,...,DISTRICT ATTORNEY REJECT,13900 SATICOY ST,,34.2104,-118.4355,POINT (-118.4355 34.2104),,,,
2,6298541,BOOKING,12/26/2021 12:00:00 AM,1445,21,Topanga,2118,25,M,H,...,CITY ATTORNEY REJECT,20100 ROSCOE BL,,34.2202,-118.5714,POINT (-118.5714 34.2202),12/26/2021 12:00:00 AM,1919,VALLEY - JAIL DIV,4279
3,5928943,BOOKING,05/09/2020 12:00:00 AM,1730,06,Hollywood,0642,35,M,B,...,CITY ATTORNEY REJECT,7600 W SUNSET BL,,34.0981,-118.355,POINT (-118.355 34.0981),05/09/2020 12:00:00 AM,1900,HOLLYWOOD,4206
4,221413759,RFC,06/29/2022 12:00:00 AM,0130,14,Pacific,1431,36,M,W,...,MISDEMEANOR COMPLAINT FILED,OCEAN FRONT WK,MARKET ST,33.9874,-118.4741,POINT (-118.4741 33.9874),,,,


In [63]:
if table_name in df_show.values:
    ingest_create_athena_table_arrests_passed = True

In [64]:
%store ingest_create_athena_table_arrests_passed

Stored 'ingest_create_athena_table_arrests_passed' (bool)


In [73]:
#AWS Glue Catalog
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://us-east-1.console.aws.amazon.com/glue/home?region=us-east-1#/v2/data-catalog/databases/view/crime_foreclosure_db?catalogId=643862913351">AWS Glue Catalog</a></b>'.format(
            region
        )
    )
)



/tmp/ipykernel_151/3782172119.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# EDA

In [66]:
!pip install awswrangler

  Using cached awswrangler-3.11.0-py3-none-any.whl.metadata (17 kB)
Using cached awswrangler-3.11.0-py3-none-any.whl (379 kB)


In [74]:
#import libraries for EDA
import awswrangler as wr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Arrest Data

In [67]:
# Read the CSV file directly from S3
df_arrest = wr.s3.read_csv("s3://sagemaker-us-east-1-643862913351/ads508-group7/arrests/Arrest_Data_from_2020_to_Present.csv")

# Display the first few rows
df_arrest.head()

[03/21/25 01:31:16] INFO     Initializing a Ray instance                                               ]8;id=752719;file:///opt/conda/lib/python3.11/site-packages/awswrangler/distributed/ray/_core.py\_core.py]8;;\:]8;id=673289;file:///opt/conda/lib/python3.11/site-packages/awswrangler/distributed/ray/_core.py#202\202]8;;\

2025-03-21 01:31:19,024	WARNING services.py:2022 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 411021312 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=0.80gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-03-21 01:31:20,328	INFO worker.py:1786 -- Started a local Ray instance.


,Report ID,Report Type,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,...,Disposition Description,Address,Cross Street,LAT,LON,Location,Booking Date,Booking Time,Booking Location,Booking Location Code
0,6636966,BOOKING,07/06/2023 12:00:00 AM,2250.0,8,West LA,817,46,M,B,...,MISDEMEANOR COMPLAINT FILED,900 GAYLEY AV,NaN,34.0637,-118.4482,POINT (-118.4482 34.0637),07/07/2023 12:00:00 AM,143.0,METRO - JAIL DIVISION,4273.0
1,6637119,BOOKING,07/07/2023 12:00:00 AM,1000.0,3,Southwest,396,39,M,B,...,MISDEMEANOR COMPLAINT FILED,40TH PL,VERMONT,34.0100,-118.2915,POINT (-118.2915 34.01),07/07/2023 12:00:00 AM,1156.0,77TH ST,4212.0
2,6624479,BOOKING,06/15/2023 12:00:00 AM,1850.0,7,Wilshire,724,33,F,H,...,MISDEMEANOR COMPLAINT FILED,100 THE GROVE DR,NaN,34.0736,-118.3563,POINT (-118.3563 34.0736),06/15/2023 12:00:00 AM,2251.0,77TH ST,4212.0
3,6636128,BOOKING,07/05/2023 12:00:00 AM,1550.0,2,Rampart,218,30,F,B,...,MISDEMEANOR COMPLAINT FILED,1000 ECHO PARK AV,NaN,34.0741,-118.2590,POINT (-118.259 34.0741),07/05/2023 12:00:00 AM,1940.0,METRO - JAIL DIVISION,4273.0
4,6636650,BOOKING,07/06/2023 12:00:00 AM,1335.0,12,77th Street,1258,31,M,H,...,NaN,7800 S BROADWAY,NaN,33.9689,-118.2783,POINT (-118.2783 33.9689),07/06/2023 12:00:00 AM,1345.0,77TH ST,4212.0


### Basic Data Information

In [75]:
df_arrest.shape

(338332, 25)

In [77]:
#are there duplicates?
df_arrest.duplicated().sum()

0

In [79]:
#get datatypes
df_arrest.dtypes

Report ID                     int64
Report Type                  object
Arrest Date                  object
Time                        float64
Area ID                       int64
Area Name                    object
Reporting District            int64
Age                           int64
Sex Code                     object
Descent Code                 object
Charge Group Code           float64
Charge Group Description     object
Arrest Type Code             object
Charge                       object
Charge Description           object
Disposition Description      object
Address                      object
Cross Street                 object
LAT                         float64
LON                         float64
Location                     object
Booking Date                 object
Booking Time                float64
Booking Location             object
Booking Location Code       float64
dtype: object

### Data Quality Report - Continous

In [80]:
#identify continuous features
conf = df_arrest.select_dtypes(include=['float64', 'int64']).columns.tolist()
conf

['Report ID',
 'Time',
 'Area ID',
 'Reporting District',
 'Age',
 'Charge Group Code',
 'LAT',
 'LON',
 'Booking Time',
 'Booking Location Code']

In [81]:
dqr_conf = pd.DataFrame({
    'Feature': conf,
    'Count': df_arrest[conf].count().values,
    'Missing Values': df_arrest[conf].isnull().sum().values,
    'Cardinality': df_arrest[conf].nunique().values,
    'Min': df_arrest[conf].min().values,
    '1st Quartile': df_arrest[conf].quantile(0.25).values,
    'Mean': df_arrest[conf].mean().values,
    'Median': df_arrest[conf].median().values,
    '3rd Quartile': df_arrest[conf].quantile(0.75).values,
    'Max': df_arrest[conf].max().values,
    'Standard Deviation': df_arrest[conf].std().values,
})
dqr_conf

,Feature,Count,Missing Values,Cardinality,Min,1st Quartile,Mean,Median,3rd Quartile,Max,Standard Deviation
0,Report ID,338332,0,338332,1.062984e+06,6.223094e+06,5.466523e+07,6.584762e+06,6.938162e+06,2.520041e+08,9.036857e+07
1,Time,338311,21,1440,1.000000e+00,9.150000e+02,1.352396e+03,1.430000e+03,1.900000e+03,2.400000e+03,6.531793e+02
2,Area ID,338332,0,21,1.000000e+00,5.000000e+00,1.047322e+01,1.100000e+01,1.600000e+01,2.100000e+01,6.278694e+00
3,Reporting District,338332,0,1281,1.000000e+02,5.130000e+02,1.093766e+03,1.149000e+03,1.601000e+03,2.199000e+03,6.271921e+02
4,Age,338332,0,96,0.000000e+00,2.600000e+01,3.521100e+01,3.300000e+01,4.200000e+01,9.600000e+01,1.230836e+01
5,Charge Group Code,306171,32161,29,1.000000e+00,6.000000e+00,1.333374e+01,1.200000e+01,2.200000e+01,9.900000e+01,7.569706e+00
6,LAT,338332,0,4884,0.000000e+00,3.400960e+01,3.361040e+01,3.405730e+01,3.417210e+01,3.433430e+01,3.961764e+00
7,LON,338332,0,4639,-1.186676e+02,-1.184313e+02,-1.167305e+02,-1.183092e+02,-1.182723e+02,0.000000e+00,1.375469e+01
8,Booking Time,262928,75404,1438,1.000000e+00,5.150000e+02,1.258309e+03,1.353000e+03,1.939000e+03,2.359000e+03,7.540066e+02
9,Booking Location Code,262924,75408,33,4.201000e+03,4.212000e+03,4.250837e+03,4.273000e+03,4.279000e+03,4.283000e+03,3.185549e+01


### Data Quality Report - Categorical

In [89]:
#identify categorical features
catf = df_arrest.select_dtypes(include=['object']).columns.tolist()
catf

['Report Type',
 'Arrest Date',
 'Area Name',
 'Sex Code',
 'Descent Code',
 'Charge Group Description',
 'Arrest Type Code',
 'Charge',
 'Charge Description',
 'Disposition Description',
 'Address',
 'Cross Street',
 'Location',
 'Booking Date',
 'Booking Location']

In [90]:
#create list to store modes and frequencies
modes = []
mode_freqs = []
second_modes = []
second_mode_freqs = []
mode_percentages = []
second_mode_percentages = []

In [91]:
# Calculate mode and frequency for each categorical feature
for feature in catf:
    count = df_arrest[feature].count()
    mode = df_arrest[feature].mode().iloc[0]
    mode_freq = df_arrest[feature].value_counts().iloc[0]
    modes.append(mode)
    mode_freqs.append(mode_freq)
    mode_percentages.append((mode_freq / count) * 100 if count > 0 else 0)

    # Calculate second mode and its frequency
    if len(df_arrest[feature].value_counts()) > 1:
        second_mode = df_arrest[feature].value_counts().index[1]
        second_mode_freq = df_arrest[feature].value_counts().iloc[1]
    else:
        second_mode = None
        second_mode_freq = 0

    second_modes.append(second_mode)
    second_mode_freqs.append(second_mode_freq)
    second_mode_percentages.append((second_mode_freq / count) * 100 if count > 0 else 0)

In [92]:
#build quality report table
# Build the quality report DataFrame
dqr_catf = pd.DataFrame({
    'Feature': catf,
    'Count': df_arrest[catf].count().values,
    'Missing Values': df_arrest[catf].isnull().sum().values,
    'Cardinality': df_arrest[catf].nunique().values,
    'Mode': modes,
    'Mode Frequency': mode_freqs,
    'Mode %': mode_percentages,
    '2nd Mode': second_modes,
    '2nd Mode Frequency': second_mode_freqs,
    '2nd Mode %': second_mode_percentages,
})
dqr_catf

,Feature,Count,Missing Values,Cardinality,Mode,Mode Frequency,Mode %,2nd Mode,2nd Mode Frequency,2nd Mode %
0,Report Type,338332,0,2,BOOKING,262928,77.713016,RFC,75404,22.286984
1,Arrest Date,338332,0,1887,06/22/2023 12:00:00 AM,426,0.125912,06/01/2020 12:00:00 AM,362,0.106995
2,Area Name,338332,0,21,Central,27054,7.996288,Rampart,26842,7.933627
3,Sex Code,338332,0,2,M,269431,79.635092,F,68901,20.364908
4,Descent Code,338332,0,19,H,173742,51.352518,B,91854,27.149073
5,Charge Group Description,306117,32215,27,Miscellaneous Other Violations,45967,15.016154,Aggravated Assault,42702,13.949568
6,Arrest Type Code,338331,1,7,F,156816,46.349876,M,149969,44.326119
7,Charge,338332,0,4922,273.5(A)PC,20077,5.934112,23152(A)VC,19875,5.874407
8,Charge Description,306171,32161,1439,CORPORAL INJURY ON SPOUSE/COHABITANT/ETC,20077,6.557447,DRUNK DRIVING ALCOHOL/DRUGS,19749,6.450317
9,Disposition Description,307256,31076,22,MISDEMEANOR COMPLAINT FILED,172804,56.241050,FELONY COMPLAINT FILED,69069,22.479301
